# Audio Transcription with Assembly AI API

In [32]:
import requests
import pandas as pd
import numpy as np
import time
import json
from IPython.display import Image

#Visualization
from wordcloud import WordCloud, STOPWORDS
import plotly.express as px

#Panel/hvplot (holoviz)
import panel as pn
pn.extension()
import param
import hvplot.pandas

#Others
import pickle
from io import StringIO

In [33]:
#need jupyter_bokeh to run panel in VSCode
import jupyter_bokeh

In [34]:
#importing personal AssemblyAI key
from Lib.API_secret import API_key

In [37]:
endpoint = "https://api.assemblyai.com/v2/transcript"
json = {
    "audio_url": "https://github.com/elitay152/AssemblyAI_Audio_Project/blob/main/286-unreasonable-f-strings.mp3?raw=True",
    "auto_highlights": True,
    "iab_categories": True,
    "sentiment_analysis": True,
    "auto_chapters": True
}
headers = {
    "authorization": API_key,
}
response = requests.post(endpoint, json=json, headers=headers)
print(response.json())

{'id': 'ralnj7l1ul-978d-49d5-820b-136c77c719a3', 'language_model': 'assemblyai_default', 'acoustic_model': 'assemblyai_default', 'language_code': 'en_us', 'status': 'queued', 'audio_url': 'https://github.com/elitay152/AssemblyAI_Audio_Project/blob/main/286-unreasonable-f-strings.mp3?raw=True', 'text': None, 'words': None, 'utterances': None, 'confidence': None, 'audio_duration': None, 'punctuate': True, 'format_text': True, 'dual_channel': None, 'webhook_url': None, 'webhook_status_code': None, 'webhook_auth': False, 'webhook_auth_header_name': None, 'speed_boost': False, 'auto_highlights_result': None, 'auto_highlights': True, 'audio_start_from': None, 'audio_end_at': None, 'word_boost': [], 'boost_param': None, 'filter_profanity': False, 'redact_pii': False, 'redact_pii_audio': False, 'redact_pii_audio_quality': None, 'redact_pii_policies': None, 'redact_pii_sub': None, 'speaker_labels': False, 'content_safety': False, 'iab_categories': True, 'content_safety_labels': {}, 'iab_categor

In [38]:
result_endpoint = endpoint + "/" + response.json()["id"]
headers_auth = {
    "authorization": API_key,
}
transcript_response = requests.get(result_endpoint, headers=headers_auth)
print("Transcription text: ", 
transcript_response.json()['text'])

#while loop for requesting transcription
while response.json()['status'] != 'completed':
    response = requests.get(result_endpoint, headers=headers_auth)
    time.sleep(3)

Transcription text:  Hello, and welcome to Python Bytes, where we deliver Python news and headlines directly to your earbuds. This is episode 286, recorded May 31, 2022. I'm Michael Kennedy. And I'm Brian Akin. And this episode is brought to you by us. If you are looking to learn Python, check out all the courses over at Talk Python Training. If you want to get better with testing, check out Brian's Pytest book, second edition. Yes. Yeah. Yes, indeed. Indeed. All right, well, let's jump right into it, Brian. Okay. Actually, about the gill, I do want to talk about the gill, but I was just realizing we're at 286. I just started watching Big Bang Theory with my youngest kid, and and there's 279 episodes of them. So we have more than it went on forever, and we have more episodes than them. That's quite a milestone. I mean, you divide that by 52, that's a lot of years. We've been doing this for a while. Yeah. So python and the gill. So the gill is a thing, right? It's like something everybo

In [82]:
output_list = ['id', 'status', 'text', 'sentiment_analysis_results', 'summary']

for i in output_list:
    print(f'{i}: ', transcript_response.json()[i])

id:  ralnj7l1ul-978d-49d5-820b-136c77c719a3
status:  completed
text:  Hello, and welcome to Python Bytes, where we deliver Python news and headlines directly to your earbuds. This is episode 286, recorded May 31, 2022. I'm Michael Kennedy. And I'm Brian Akin. And this episode is brought to you by us. If you are looking to learn Python, check out all the courses over at Talk Python Training. If you want to get better with testing, check out Brian's Pytest book, second edition. Yes. Yeah. Yes, indeed. Indeed. All right, well, let's jump right into it, Brian. Okay. Actually, about the gill, I do want to talk about the gill, but I was just realizing we're at 286. I just started watching Big Bang Theory with my youngest kid, and and there's 279 episodes of them. So we have more than it went on forever, and we have more episodes than them. That's quite a milestone. I mean, you divide that by 52, that's a lot of years. We've been doing this for a while. Yeah. So python and the gill. So the gi

In [40]:
#save pickle
with open('speech_data.pkl', 'wb') as f:
    pickle.dump(transcript_response.json().copy(), f)

## Dashboard components

In [41]:
#load data pickle
with open('speech_data.pkl', 'rb') as f:
    data = pickle.load(f)

### Download transcript widget

In [42]:
buffer = StringIO()
buffer.write(data['text'])
buffer.seek(0)

0

In [43]:
transcript_download = pn.widgets.FileDownload(file=buffer, filename='transcript.txt', button_type='success')
transcript_download

BokehModel(combine_events=True, render_bundle={'docs_json': {'1c872276-317c-4ad6-a267-51b0b4452d43': {'defs': …

### Audio Play 

In [59]:
audio_url = "https://github.com/elitay152/AssemblyAI_Audio_Project/blob/main/286-unreasonable-f-strings.mp3?raw=True"
audio_play = pn.pane.Audio(audio_url, name='Audio', time=0)
audio_play

BokehModel(combine_events=True, render_bundle={'docs_json': {'2e451a46-0771-43a6-ad6e-1a8d55164bda': {'defs': …

### Sentiment Plot

In [45]:
sentiment = data['sentiment_analysis_results']

In [46]:
sentiment_df = pd.DataFrame(sentiment)
sentiment_df

,text,start,end,sentiment,confidence,speaker
0,"Hello, and welcome to Python Bytes, where we d...",410,5530,POSITIVE,0.828239,None
1,"This is episode 286, recorded May 31, 2022.",5610,10974,NEUTRAL,0.951202,None
2,I'm Michael Kennedy.,11012,11914,NEUTRAL,0.852055,None
3,And I'm Brian Akin.,11962,12954,NEUTRAL,0.791588,None
4,And this episode is brought to you by us.,13002,15038,POSITIVE,0.518970,None
...,...,...,...,...,...,...
432,"Thanks, everybody, for coming.",1577442,1578760,POSITIVE,0.972476,None
433,"Bye, everyone.",1579130,1579798,NEUTRAL,0.599049,None
434,Thanks for being here.,1579884,1580566,POSITIVE,0.949438,None
435,Bye.,1580588,1580962,NEUTRAL,0.513745,None


In [47]:
sentiment_df_grouped = sentiment_df['sentiment'].value_counts()
sentiment_df_grouped

NEUTRAL     278
POSITIVE    130
NEGATIVE     29
Name: sentiment, dtype: int64

In [75]:
# bar plot
sentiment_plot = sentiment_df_grouped.hvplot(title='Sentences by Sentiment Category', kind='bar', c='sentiment')
pn.Row(sentiment_plot)

BokehModel(combine_events=True, render_bundle={'docs_json': {'171f9934-bfc7-47f3-9cdd-30b89193031d': {'defs': …

In [76]:
positive_df = sentiment_df[sentiment_df['sentiment']=='POSITIVE'][['text', 'sentiment']]
negative_df = sentiment_df[sentiment_df['sentiment']=='NEGATIVE'][['text', 'sentiment']]
neutral_df = sentiment_df[sentiment_df['sentiment']=='NEUTRAL'][['text', 'sentiment']]

sentiment_tabs = pn.Tabs(('Sentiment overview', sentiment_plot), \
    ('Positive', pn.widgets.DataFrame(positive_df, autosize_mode='fit_columns', width=700, height=300)),\
         ('Negative', pn.widgets.DataFrame(negative_df, autosize_mode='fit_columns', width=700, height=300)),\
            ('Neutral', pn.widgets.DataFrame(neutral_df, autosize_mode='fit_columns', width=700, height=300)))

sentiment_tabs

BokehModel(combine_events=True, render_bundle={'docs_json': {'6ee3765e-9fd5-4433-a529-3ee0fb48bb71': {'defs': …

### Word cloud

In [50]:
stopwords = set(STOPWORDS)

In [51]:
transcript = data['text']

In [52]:
transcript_lower = [item.lower() for item in str(transcript).split()]
transcript_lower

['hello,',
 'and',
 'welcome',
 'to',
 'python',
 'bytes,',
 'where',
 'we',
 'deliver',
 'python',
 'news',
 'and',
 'headlines',
 'directly',
 'to',
 'your',
 'earbuds.',
 'this',
 'is',
 'episode',
 '286,',
 'recorded',
 'may',
 '31,',
 '2022.',
 "i'm",
 'michael',
 'kennedy.',
 'and',
 "i'm",
 'brian',
 'akin.',
 'and',
 'this',
 'episode',
 'is',
 'brought',
 'to',
 'you',
 'by',
 'us.',
 'if',
 'you',
 'are',
 'looking',
 'to',
 'learn',
 'python,',
 'check',
 'out',
 'all',
 'the',
 'courses',
 'over',
 'at',
 'talk',
 'python',
 'training.',
 'if',
 'you',
 'want',
 'to',
 'get',
 'better',
 'with',
 'testing,',
 'check',
 'out',
 "brian's",
 'pytest',
 'book,',
 'second',
 'edition.',
 'yes.',
 'yeah.',
 'yes,',
 'indeed.',
 'indeed.',
 'all',
 'right,',
 'well,',
 "let's",
 'jump',
 'right',
 'into',
 'it,',
 'brian.',
 'okay.',
 'actually,',
 'about',
 'the',
 'gill,',
 'i',
 'do',
 'want',
 'to',
 'talk',
 'about',
 'the',
 'gill,',
 'but',
 'i',
 'was',
 'just',
 'realizin

In [53]:
all_words = ' '.join(transcript_lower)
all_words

"hello, and welcome to python bytes, where we deliver python news and headlines directly to your earbuds. this is episode 286, recorded may 31, 2022. i'm michael kennedy. and i'm brian akin. and this episode is brought to you by us. if you are looking to learn python, check out all the courses over at talk python training. if you want to get better with testing, check out brian's pytest book, second edition. yes. yeah. yes, indeed. indeed. all right, well, let's jump right into it, brian. okay. actually, about the gill, i do want to talk about the gill, but i was just realizing we're at 286. i just started watching big bang theory with my youngest kid, and and there's 279 episodes of them. so we have more than it went on forever, and we have more episodes than them. that's quite a milestone. i mean, you divide that by 52, that's a lot of years. we've been doing this for a while. yeah. so python and the gill. so the gill is a thing, right? it's like something everybody knows about, i th

In [54]:
#word cloud plot
wordcloud = WordCloud(background_color='black', stopwords=stopwords, max_words=20, \
    colormap='viridis', collocations=False).generate(all_words)

wordcloud_plot = px.imshow(wordcloud)
#remove labels on axes
wordcloud_plot.update_xaxes(showticklabels=False)
wordcloud_plot.update_yaxes(showticklabels=False)
wordcloud_plot

In [79]:
#Create interactive slider
class Controller(param.Parameterized):
    word_slider = param.Integer(30, bounds=(5, 50), step=5)

controller = Controller()

@pn.depends(controller.param.word_slider)
def update_wordcloud(num_words):
    #word cloud plot
    wordcloud = WordCloud(background_color='black', stopwords=stopwords, max_words=20, \
        colormap='viridis', collocations=False).generate(all_words)
    wordcloud_plot = px.imshow(wordcloud)
    #remove labels on axes
    wordcloud_plot.update_xaxes(showticklabels=False)
    wordcloud_plot.update_yaxes(showticklabels=False)
    return wordcloud_plot

### Auto chapter summary

In [56]:
chapters = data['chapters']
chapters

[{'summary': "This is episode 286 of Python Bytes. We deliver Python news and headlines directly to your earbuds. If you are looking to learn Python, check out all the courses over at Talk Python Training. Want to get better with testing? Check out Brian's Pytest book, second edition.",
  'gist': 'Python Bytes: Episode 286',
  'headline': 'This is episode 286 of Python Bytes',
  'start': 410,
  'end': 29714},
 {'summary': "There's 279 episodes of Big Bang Theory. That's quite a milestone. You divide that by 52, that's a lot of years. We've been doing this for a while.",
  'gist': "The Big Bang Theory's 286 Episodes",
  'headline': "Big Bang Theory now has 286 episodes. That's quite a milestone",
  'start': 29762,
  'end': 53086},
 {'summary': 'An article called The Python Gill Past, Present, and Future talks about some of the no gill options that are coming. Does talk through reference counting, talks through the advantages of having it around. Predicts speed ups in the next few years.

In [73]:
chapter_summary = pn.widgets.StaticText(value=chapters[0]['summary'], width=1000, sizing_mode='scale_height')
chapter_summary

BokehModel(combine_events=True, render_bundle={'docs_json': {'3b4dffe9-39f8-4ea3-a729-c4fd9985b73e': {'defs': …

In [58]:
button = pn.widgets.Button(name=str(int(chapters[0]['start']/1000)), button_type='primary')
button

BokehModel(combine_events=True, render_bundle={'docs_json': {'49c4c9cf-8da5-4a1d-82bc-ee00831c8d07': {'defs': …

In [60]:
chapter_audio = pn.pane.Audio(audio_url, name='Audio', time=round(chapters[0]['start']/1000))
chapter_audio

BokehModel(combine_events=True, render_bundle={'docs_json': {'4bc92d9a-6ff7-4048-8d65-236c144f5ff2': {'defs': …

In [71]:
#create chapter summary layout
chapters_layout = pn.Column(pn.pane.Markdown('### Auto Chapter Summary'))

class ButtonAudio():
    def __init__(self, start_time):
        self.start_time = start_time
        self.button = pn.widgets.Button(name=str(int(self.start_time/1000)), button_type='primary', width=60)
        self.chapter_audio = pn.pane.Audio(audio_url, name='Audio', time=round(self.start_time/1000))
        self.button.on_click(self.move_audio_head)
    
    def move_audio_head(self, event):
        self.chapter_audio.time = self.start_time/1000

for chapter in chapters:
    chapter_summary = pn.widgets.StaticText(value=chapter['summary'], width=1000, sizing_mode='scale_height')
    button_audio = ButtonAudio(chapter['start'])
    button = button_audio.button
    chapter_audio = button_audio.chapter_audio
    chapters_layout.append(pn.Row(pn.Column(button), pn.Column(chapter_audio), pn.Column(chapter_summary)))

chapters_layout

BokehModel(combine_events=True, render_bundle={'docs_json': {'779603cd-2e25-4ac3-b493-86f440f00fa0': {'defs': …

### Auto Highlights

In [65]:
highlights = data['auto_highlights_result']['results']
highlights_df = pd.DataFrame(highlights)
highlights_df

,count,rank,text,timestamps
0,2,0.08,Run Python,"[{'start': 420840, 'end': 421566}, {'start': 4..."
1,1,0.08,Python web apps,"[{'start': 1351126, 'end': 1352006}]"
2,1,0.07,Python interpreters,"[{'start': 410276, 'end': 411226}]"
3,1,0.07,Python talks,"[{'start': 178220, 'end': 179026}]"
4,1,0.07,Python news,"[{'start': 2698, 'end': 3166}]"
5,1,0.07,Python Bytes,"[{'start': 1492, 'end': 2106}]"
6,1,0.07,main Python,"[{'start': 192976, 'end': 194582}]"
7,1,0.07,Python client side,"[{'start': 1427516, 'end': 1428742}]"
8,1,0.07,Talk Python Training,"[{'start': 17764, 'end': 18686}]"
9,1,0.07,Python pi Script WebAssembly,"[{'start': 1349526, 'end': 1351062}]"


In [66]:
highlights_df_grouped = highlights_df.groupby(['count', 'rank'])['text'].apply(', \n'.join).reset_index()
highlights_df_grouped

,count,rank,text
0,1,0.05,"PyCharm Vs code PyCharm, \nUI stuff"
1,1,0.06,"regular expression stuff, \nother things"
2,1,0.07,"Python interpreters, \nPython talks, \nPython ..."
3,1,0.08,Python web apps
4,2,0.05,other people
5,2,0.08,Run Python


In [80]:
#scatter plot
highlights_plot = highlights_df_grouped.hvplot.points(x='count', y='rank', padding=0.4, hover_cols='all', width=1300, height=600, size=50, title='Automatic Highlights') *\
highlights_df_grouped.hvplot.labels(x='count', y='rank', text='text', text_baseline='top', hover=False).opts(fontscale=1.5)
pn.Row(highlights_plot)

BokehModel(combine_events=True, render_bundle={'docs_json': {'8f35c96a-5018-40a0-bc03-0ac37e26a311': {'defs': …

### Dashboard

In [81]:
#dashboard template
template = pn.template.FastListTemplate(
    title = 'Audio Content Exploration Dash', 
    sidebar = [pn.pane.Markdown('# Explore Audio Content'),
    pn.pane.Markdown('#### This app analyzes the content of your audio file, including sentiment, wordcloud, automatic content summary, and highlights'),
    pn.pane.Markdown('#### This example is based on the audio content of Episode 286: "Unreasonable f-strings", from the podcast PythonBytes by Michael Kennedy and Brian Okken'),
    pn.pane.Markdown('### [Link to podcast episode!](https://pythonbytes.fm/episodes/show/286/unreasonable-f-strings)'),
    pn.pane.Markdown('### Download transcript:'),
    transcript_download
    ],
main = [pn.Row(pn.Column(sentiment_tabs), pn.Column(pn.Row(controller.param.word_slider),
    pn.Row(update_wordcloud))
    ), 
    pn.Row(chapters_layout),
    pn.Row(highlights_plot)],
accent_base_color = '#88d8b0',
header_background = '#c0b9dd',
)

template.show()

Launching server at http://localhost:61042
